In [1]:
import glob
import matplotlib.pyplot as plt

import tensorflow
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import cv2

/Users/lorenzocesconetto/Desktop/ITAU-ANALYTICS/Skin-Cancer/skin_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/lorenzocesconetto/Desktop/ITAU-ANALYTICS/Skin-Cancer/skin_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/lorenzocesconetto/Desktop/ITAU-ANALYTICS/Skin-Cancer/skin_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)ty

In [2]:
mel_imgs_name = glob.glob('../data_processed/test/mel/*.jpg')
non_mel_imgs_name = glob.glob('../data_processed/test/other/*.jpg')

In [3]:
mel_name_2_dim = {x.split('/')[-1]: plt.imread('../ISIC-2019/ISIC_2019_Training_Input/' + 
                                           x.split('/')[-1]).shape for x in mel_imgs_name}

In [4]:
non_mel_name_2_dim = {x.split('/')[-1]: plt.imread('../ISIC-2019/ISIC_2019_Training_Input/' + 
                                           x.split('/')[-1]).shape for x in non_mel_imgs_name}

### 0. Dark padding - All images

In [5]:
vgg = load_model('../saved_models/VGG_v1_auc_0.897.h5', compile=False)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


#### 0.1. Melanoma

In [6]:
mel_shape_2_scores = {x: [] for x in list(set(mel_name_2_dim.values()))}

for img_name in mel_imgs_name:
    img = tensorflow.keras.applications.vgg16.preprocess_input(plt.imread(img_name))
    img = np.expand_dims(img, axis=0)
    mel_shape_2_scores[mel_name_2_dim[img_name.split('/')[-1]]].append(vgg.predict(img))

In [7]:
mel_shape_2_statistics = {}
for key, value in mel_shape_2_scores.items():
    avg = np.mean(value)
    std = np.std(value)
    mel_shape_2_statistics[key] = {'total': len(value), 'avg': avg, 'std': std }

In [8]:
df = pd.DataFrame(mel_shape_2_statistics).transpose()
df = df[df['total'] >= 20]
df.sort_values('avg', ascending=False)

,,,total,avg,std
1024,1024,3,389.0,0.780731,0.245087
768,1024,3,20.0,0.683007,0.240395
450,600,3,168.0,0.681813,0.269838
680,1024,3,39.0,0.526021,0.246804


#### 0.2. Non-Melanoma

In [10]:
non_mel_shape_2_scores = {x: [] for x in list(set(non_mel_name_2_dim.values()))}

for img_name in non_mel_imgs_name:
    img = tensorflow.keras.applications.vgg16.preprocess_input(plt.imread(img_name))
    img = np.expand_dims(img, axis=0)
    non_mel_shape_2_scores[non_mel_name_2_dim[img_name.split('/')[-1]]].append(vgg.predict(img))

In [12]:
non_mel_shape_2_statistics = {}
for key, value in non_mel_shape_2_scores.items():
    avg = np.mean(value)
    std = np.std(value)
    non_mel_shape_2_statistics[key] = {'total': len(value), 'avg': avg, 'std': std }

In [13]:
df = pd.DataFrame(non_mel_shape_2_statistics).transpose()
df = df[df['total'] >= 20]
df.sort_values('avg')

,,,total,avg,std
450,600,3,1321.0,0.242627,0.225292
685,1024,3,25.0,0.280906,0.168015
680,1024,3,124.0,0.287031,0.173685
768,1024,3,86.0,0.411970,0.207259
1024,1024,3,1372.0,0.438546,0.294494


### 1. Restricted dark padding

In [2]:
vgg = load_model('../saved_models/VGG_v7_auc_0.901.h5', compile=False)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


#### 1.1. Melanoma

In [63]:
shape_2_scores = {x: [] for x in list(set(name_2_dim.values()))}

for img_name in imgs_name:
    img = tensorflow.keras.applications.vgg16.preprocess_input(plt.imread(img_name))
    img = np.expand_dims(img, axis=0)
    shape_2_scores[name_2_dim[img_name.split('/')[-1]]].append(vgg.predict(img))

In [64]:
shape_2_statistics = {}
for key, value in shape_2_scores.items():
    avg = np.mean(value)
    std = np.std(value)
    shape_2_statistics[key] = {'total': len(value), 'avg': avg, 'std': std }

In [65]:
df = pd.DataFrame(shape_2_statistics).transpose()
df = df[df['total'] >= 20]
df

,,,total,avg,std
450,600,3,168.0,0.826100,0.230404
768,1024,3,20.0,0.806100,0.230702
1024,1024,3,389.0,0.846299,0.218769
680,1024,3,39.0,0.562991,0.317695


#### 1.2. Non-Melanoma

In [6]:
shape_2_scores = {x: [] for x in list(set(name_2_dim.values()))}

for img_name in imgs_name:
    img = tensorflow.keras.applications.vgg16.preprocess_input(plt.imread(img_name))
    img = np.expand_dims(img, axis=0)
    shape_2_scores[name_2_dim[img_name.split('/')[-1]]].append(vgg.predict(img))

In [7]:
shape_2_statistics = {}
for key, value in shape_2_scores.items():
    avg = np.mean(value)
    std = np.std(value)
    shape_2_statistics[key] = {'total': len(value), 'avg': avg, 'std': std }

In [8]:
df = pd.DataFrame(shape_2_statistics).transpose()
df = df[df['total'] >= 20]
df.sort_values('avg')

,,,total,avg,std
450,600,3,1321.0,0.222527,0.282630
685,1024,3,25.0,0.225676,0.249091
680,1024,3,124.0,0.238784,0.230302
768,1024,3,86.0,0.450418,0.293636
1024,1024,3,1372.0,0.462112,0.348896


### 2. Resizing and deforming images

In [15]:
# Resizing VGG_v10_auc_0.877.h5
vgg = load_model('../saved_models/VGG_v10_auc_0.890.h5', compile=False)

#### 2.1. Melanoma

In [9]:
shape_2_scores = {}
name_2_dim = {}
shapes = set()

for img_name in imgs_name:
    img = plt.imread('../ISIC-2019/ISIC_2019_Training_Input/' + img_name)
    img_shape = img.shape
    name_2_dim[img_name] = img_shape
    
    img = cv2.resize(img, (224, 224))
    img = tensorflow.keras.applications.vgg16.preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    score = vgg.predict(img)
    
    if img_shape in shapes:
        shape_2_scores[img_shape].append(score)
    else:
        shapes.add(img_shape)
        shape_2_scores[img_shape] = [score]
    

In [12]:
shape_2_statistics = {}
for key, value in shape_2_scores.items():
    avg = np.mean(value)
    std = np.std(value)
    shape_2_statistics[key] = {'total': len(value), 'avg': avg, 'std': std }

In [13]:
df = pd.DataFrame(shape_2_statistics).transpose()
df = df[df['total'] >= 20]
df

,,,total,avg,std
1024,1024,3,389.0,0.787053,0.237618
450,600,3,168.0,0.737229,0.248183
680,1024,3,39.0,0.538376,0.248920
768,1024,3,20.0,0.703303,0.246242


#### 2.2. Non-Melanoma

In [17]:
shape_2_scores = {}
name_2_dim = {}
shapes = set()

for img_name in imgs_name:
    img = plt.imread('../ISIC-2019/ISIC_2019_Training_Input/' + img_name)
    img_shape = img.shape
    name_2_dim[img_name] = img_shape
    
    img = cv2.resize(img, (224, 224))
    img = tensorflow.keras.applications.vgg16.preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    score = vgg.predict(img)
    
    if img_shape in shapes:
        shape_2_scores[img_shape].append(score)
    else:
        shapes.add(img_shape)
        shape_2_scores[img_shape] = [score]
    

In [18]:
shape_2_statistics = {}
for key, value in shape_2_scores.items():
    avg = np.mean(value)
    std = np.std(value)
    shape_2_statistics[key] = {'total': len(value), 'avg': avg, 'std': std }

In [20]:
df = pd.DataFrame(shape_2_statistics).transpose()
df = df[df['total'] >= 20]
df.sort_values('avg')

,,,total,avg,std
450,600,3,1321.0,0.256238,0.281971
680,1024,3,124.0,0.276617,0.203529
685,1024,3,25.0,0.318228,0.214278
768,1024,3,86.0,0.347700,0.257404
1024,1024,3,1372.0,0.393807,0.309277


### 3. Only Squared (1024 x 1024)

In [14]:
vgg = load_model('../saved_models/VGG_v17_auc_0.813.h5', compile=False)

#### 3.1. Melanoma

In [15]:
mel_shape_2_scores = {x: [] for x in list(set(mel_name_2_dim.values()))}

for img_name in mel_imgs_name:
    img = tensorflow.keras.applications.vgg16.preprocess_input(plt.imread(img_name))
    img = np.expand_dims(img, axis=0)
    mel_shape_2_scores[mel_name_2_dim[img_name.split('/')[-1]]].append(vgg.predict(img))

In [16]:
mel_shape_2_statistics = {}
for key, value in mel_shape_2_scores.items():
    avg = np.mean(value)
    std = np.std(value)
    mel_shape_2_statistics[key] = {'total': len(value), 'avg': avg, 'std': std }

In [17]:
df = pd.DataFrame(mel_shape_2_statistics).transpose()
df = df[df['total'] >= 20]
df.sort_values('avg', ascending=False)

,,,total,avg,std
1024,1024,3,389.0,0.741020,0.243624
450,600,3,168.0,0.721749,0.225686
768,1024,3,20.0,0.640181,0.251827
680,1024,3,39.0,0.300992,0.201099


#### 3.2. Non-Melanoma

In [18]:
non_mel_shape_2_scores = {x: [] for x in list(set(non_mel_name_2_dim.values()))}

for img_name in non_mel_imgs_name:
    img = tensorflow.keras.applications.vgg16.preprocess_input(plt.imread(img_name))
    img = np.expand_dims(img, axis=0)
    non_mel_shape_2_scores[non_mel_name_2_dim[img_name.split('/')[-1]]].append(vgg.predict(img))

In [19]:
non_mel_shape_2_statistics = {}
for key, value in non_mel_shape_2_scores.items():
    avg = np.mean(value)
    std = np.std(value)
    non_mel_shape_2_statistics[key] = {'total': len(value), 'avg': avg, 'std': std }

In [20]:
df = pd.DataFrame(non_mel_shape_2_statistics).transpose()
df = df[df['total'] >= 20]
df.sort_values('avg')

,,,total,avg,std
680,1024,3,124.0,0.192730,0.143964
685,1024,3,25.0,0.213216,0.157001
450,600,3,1321.0,0.402167,0.244968
1024,1024,3,1372.0,0.403109,0.270736
768,1024,3,86.0,0.436448,0.251332
